In [1]:
#Importing packages we need
from numpy import genfromtxt #This is used to loading a csv-file as a numpy array
import numpy as np
import scipy.optimize as opt #used to numerically optimize
import matplotlib.pyplot as plt #pyplot is used to plot the data
import pandas as pd
from matplotlib import cm

In [2]:
#Loading data (here we use the data we simulated in our earlier script)

#Locate my folder
my_folder='C:/Users/vjule/OneDrive/Dokumenter/Diverse/Cornell/Courses/PhD_seminar_Market_microstructure_NRE5060/My_project/Data/'

x_val = genfromtxt(my_folder+'GARCH11sim.csv', delimiter=',') #loading the csv data and storing it as a np array

In [10]:
def ARCH11_filter(omega, alpha,x_values): #function to find sigma^2 from observations
    T=len(x_values)
    
    #Define the series sigma^2_t (sigma2) as empty variables we assign values to later. 
    sigma2=np.empty(T)
    
    sigma2[0]=1
    
    for t in range(1,T):
        sigma2[t]=omega + alpha*x_values[t-1]**2
    
    return sigma2

def ARCH11_likelihood(omega,alpha):
    
    sigma2=ARCH11_filter(omega, alpha, x_val)
    
    LogL=-np.sum(-np.log(sigma2)-x_val**2/sigma2) #Constants have been removed from the expression
    #Note we take the expression and multiply with (-1) as we are going to minimize the function in the optimization
    
    return LogL

def optim(theta): #defining an optimization function that can change on parameters omega and alpha
    ome,alp=theta
    return ARCH11_likelihood(ome, alp)

In [11]:
theta0=np.array([0.25,2]) #initial guesses
res=opt.minimize(optim, theta0, bounds=((0.0001,None),(0,None)))

In [14]:
#Now computing Standard errors
v_hessian=res.hess_inv.todense() #retrieves the negative inverse hessian matrix (note we have minimized the negative log likelihood function)
se_hessian=np.sqrt(np.diagonal(v_hessian))

In [19]:
#res.x gives the parameter estimate(s)
omega_hat=res.x[0]
alpha_hat=res.x[1] 

In [20]:
print('Estimates are given by with std. errors in ():')
print('omega_hat= '+str(omega_hat)+' (' +str(se_hessian[0])+')'+' ' ' -> T-val=' +str(omega_hat/se_hessian[0])+'' )
print('alpha_hat= '+str(alpha_hat)+' (' +str(se_hessian[1])+')'+' ' ' -> T-val=' +str(alpha_hat/se_hessian[1])+'' )
print('Maximized log-likelihood (without constants) ' +str(ARCH11_likelihood(omega_hat,alpha_hat)))

Estimates are given by with std. errors in ():
omega_hat= 0.9080608657571719 (0.047396575848710626)  -> T-val=19.15878625189492
alpha_hat= 0.5578816484391389 (0.04652641332333561)  -> T-val=11.990643778237036
Maximized log-likelihood (without constants) 1453.4997891749692
